In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
class BasicBlock(nn.Module):
    expansion = 1
    
    def __init__(self, in_channels, out_channels, stride = 1):
        super(BasicBlock, self).__init__()
        """
        stride => convolution step size
        bias => whether to use bias value or not
        """
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=True)
        
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1,padding=1,bias=True)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion*out_channels, kernel_size=1, stride=1, bias=False),
                # 1*1 convolution to serve in_channels, self.expansion*output_channels difference
                nn.BatchNorm2d(self.expansion * out_channels)
            )
    
    def forward(self, x):
        print(x.shape)
        print(self.bn1)
        out = F.relu(
            self.bn1(x))
        residual = out
        
        out = self.conv1(out)
        out = self.conv2(F.relu(self.bn2(out)))
        
        if hasattr(self, "shorcut"):
            residual = self.shortcut(x)
            
        out += residual
        
        
        
        return out
        

In [7]:
class PreActResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(PreActResNet, self).__init__()
        self.in_Channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0],stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1],stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2],stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3],stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
    
        
    def _make_layer(self,block,out_channels,blocks,stride = 1):
        layers = []
        layers.append(block(self.in_Channels, out_channels, 1))
        self.in_Channels = out_channels * block.expansion
        
        for _ in range(1, blocks):
            layers.append(block(self.in_Channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

In [10]:
def prac_resnet34(num_classes = 1000) :
    return PreActResNet(BasicBlock, [3, 4, 6, 3], num_classes)


model = prac_resnet34(num_classes=1000)
inputs = torch.randn(1, 3, 224, 224)
output = model(inputs)
print(output.shape)

torch.Size([1, 64, 114, 114])
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([1, 64, 114, 114])
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([1, 64, 114, 114])
BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
torch.Size([1, 64, 114, 114])
BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


RuntimeError: running_mean should contain 64 elements not 128

In [ ]:
class IncetionModule(nn.Module) : 

    def __init__(self, in_channels, out_1x1, reduce_3x3, out_3x3, reduce_5x5, out_5x5, out_pool) : 
        super(IncetionModule, self).__init__()

        # 1x1 convolution branch
        self.conv1x1 = nn.Conv2d(in_channels, out_1x1, kernel_size=1)
        # ---->>>> 입력 in_channels ->  out_1x1 변환 
        # 3x3 convolution branch
        self.conv3x3_reduce = nn.Conv2d(in_channels, reduce_3x3, kernel_size=1) # reduce : 줄이다.. 
        # ---->>>> (목적 : 입력 채널 줄이기) -> 입력 in_channels -> reduce_3x3 변환 
        self.conv3x3 = nn.Conv2d(reduce_3x3, out_3x3, kernel_size=3, padding=1)
        # ---->>>> 입력 : reduce_3x3 -> out_3x3 변환 / 출력 크기를 입력과 동일하게 유지 합니다.

 

        # 5x5 convolution branch
        self.conv5x5_reduce = nn.Conv2d(in_channels, reduce_5x5, kernel_size=1)
        self.conv5x5 = nn.Conv2d(reduce_5x5, out_5x5, kernel_size=5, padding=2)

        # Max Pooling branch 
        self.pool = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.conv_pool = nn.Conv2d(in_channels, out_pool, kernel_size=1)

    def forward(self, x) : 
        # 1x1 convolution branch
        out_1x1 = F.relu(self.conv1x1(x))

        # 3x3 convolution branch
        out_3x3 = F.relu(self.conv3x3(F.relu(self.conv3x3_reduce(x))))
        ### 순서 : self.conv3x3_reduce(x) -> relu -> conv3x3(x) -> relu

        # 5x5 convolution branch
        out_5x5 = F.relu(self.conv5x5(F.relu(self.conv5x5_reduce(x))))
        ### 순서 : self.conv5x5_reduce(x) -> relu -> self.conv5x5(x) -> relu

        # Max Pooling branch 
        out_pool = F.relu(self.conv_pool(self.pool(x)))
        ### 순서 : self.pool(x) -> self.conv_pool(self.pool(x)) -> relu

        # Concatenate all branch 
        out = torch.cat([out_1x1, out_3x3, out_5x5, out_pool], dim=1)

        return out